# LDAP Auth Method

Vault is an API driven software component aimed at automation. Thus most authentications to Vault are performed via machines, using authentication methods such as JWT or mTLS. However, Vault needs to be operated and secured by people. Moreover, it is not too uncommon to see Vault used as a "glorified shared password manager" and thus requires human users to authenticate to it.

While other solutions exist that enable SSO (e.g. OIDC or Kerberos), a very common way to authenticate human users to Vault is via LDAP. This is mostly due to the fact that its setup is very straight forward, and typically flexible enough. In this module we will have a look at LDAP authentication.

In case you are not familiar with it, LDAP is a protocol that enables you to communicate with directory servers, such as Microsoft Active Directory (AD). Such directory servers (often also called domain controllers in the Windows world) store information about entities within an organisation. They typically have an internal representation of the organisation's structure, and all its employees within that structure. Domain controllers are used to store much more than human users and org structures, but this is what we will focus on in this module.

Directory servers typically contain user information such as usernames, group membership, and a password. This enables Vault to effectively delegate authentication of a user to such a server (via LDAP). After successful authentication, the group membership can then be used to grant permissions to a user. This is very convenient, as joiner/leaver/mover processes are typically implemented on such servers, meaning that new employees are automatically onboarded to Vault as soon as they are added to some groups within the directory server.

If this is slightly unclear, either have a read on [directory services](https://en.wikipedia.org/wiki/Directory_service) and [LDAP](https://en.wikipedia.org/wiki/Lightweight_Directory_Access_Protocol), or just move on and discover it as we go along.

### Starting Vault

Let us start a Vault server. This will run Vault in the background and push the logs to `/tmp/vault.log`. If at any point in time the Vault crashes, this command will need to be used again to re-launch the Vault server.

For now, do not worry about the configuration with which we are starting the server. This will be covered in a separate module.

In [ ]:
nohup bash -c '
  vault server -dev -dev-root-token-id=root-token -dev-listen-address="0.0.0.0:8200"
' > /tmp/vault.log 2>&1 &
echo $! > /tmp/vault.pid

### Preparing the LDAP Server

Before we can started with this module, we will need to have a directory service up and running. Let us start one.

In [ ]:
docker run --detach --rm --name openldap \
  --network kind \
  -e LDAP_ADMIN_USERNAME=admin \
  -e LDAP_ADMIN_PASSWORD=admin \
  -e LDAP_ROOT=dc=example,dc=org \
  -e LDAP_ADMIN_DN=cn=admin,dc=example,dc=org \
  -p 1389:1389 \
  bitnami/openldap:2.6.9-debian-12-r16@sha256:907bfb47ef60895427018fa7c19c4d3961624d97e5ce50a41a44c2a63745771b

### Investigating the Structure

The above command started an OpenLDAP directory server with some pre-configured data for us. It configured the following:

- A base domain of `example.org`.
- An admin user with password `admin`, whose distinguished name is `cn=admin,dc=example,dc=org`. These names are essentially a unique reference to a directory element. See more information [here](https://learn.microsoft.com/en-us/previous-versions/windows/desktop/ldap/distinguished-names) if you are interested.
- Two users, `cn=user01,ou=users,dc=example,dc=org` and `cn=user02,ou=users,dc=example,dc=org` with password `bitnami1` and `bitnami2` respectively.
- A group `cn=readers,ou=groups,dc=example,dc=org` which contains both users above.

Have a look at this structure by executing the command below:

In [ ]:
# Test this when running inside the docker ... does it connect?
ldapsearch -x -H "ldap://127.0.0.1:1389" -b "dc=example,dc=org"

### Cleaning Up

At the end of each module, you should clean up your Vault instance. This is done by shutting it down and wiping its database to restore its state.

In [ ]:
kill $(cat /tmp/vault.pid)
rm /tmp/vault.log
rm /tmp/vault.pid
docker stop openldap